In [1]:
# Import necessary packages for notebook
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import warnings

In [4]:
spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

In [ ]:
raw_games = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "games") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [9]:
games = raw_games.select(raw_games.value.cast('string'))

final_schema = StructType([StructField('GameKey', StringType(), True),StructField('AwayTeam', StringType(), True),StructField('HomeTeam', StringType(), True)])


games_df = games.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [11]:
games_df.show()

+---------+--------+--------+
|  GameKey|AwayTeam|HomeTeam|
+---------+--------+--------+
|202110802|     CAR|     ATL|
|202110804|     MIA|     BUF|
|202110806|      SF|     CHI|
|202110808|     PIT|     CLE|
|202110811|     PHI|     DET|
|202110814|     TEN|     IND|
|202110824|     CIN|     NYJ|
|202110813|     LAR|     HOU|
|202110829|      NE|     LAC|
|202110830|     JAX|     SEA|
|202110810|     WAS|     DEN|
|202110822|      TB|      NO|
|202110820|     DAL|     MIN|
+---------+--------+--------+

